Script for downloading exploration data from where we saved it in our google drive

In [4]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import pandas as pd
import io
import os
from pprint import pprint as pp

def authenticate_service_account(json_keyfile_path):
    gauth = GoogleAuth()
    gauth.settings['client_config_backend'] = 'service'
    gauth.settings['service_config'] = {
        "client_json_file_path": json_keyfile_path,
        "client_user_email": None  # Set to None for service account
    }
    
    gauth.ServiceAuth()
    drive = GoogleDrive(gauth)
    return drive

def load_csv_files_from_folder(drive, folder_id):
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and mimeType='text/csv' and trashed=false"}).GetList()
    
    col_to_keep = ['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName','2024-03-31']
    directory = '../../data/exploration_data/processed'
    
    for file in file_list:
        print(f'Loading file: {file["title"]}')
        file_content = file.GetContentString()
        df = pd.read_csv(io.StringIO(file_content))
        df = df[col_to_keep]
        
        file_path = os.path.join(directory, f'Trimmed_{file["title"]}.pkl')
        df.to_pickle(file_path)


json_keyfile_path = '../../gdrive_key.json'  # private key not added to git
drive = authenticate_service_account(json_keyfile_path)
folder_id = '1GLHBoTSukbHryHSBUcoHZ848Huq9RkVE'  # not private information just our gdrive, can't be accessed without premissions
load_csv_files_from_folder(drive, folder_id)

Loading file: Metro_median_sale_price_all_homes_raw_monthly.csv
Loading file: Metro_sales_count_nowcast.csv
Loading file: Metro_mlp_raw_all_homes_monthly.csv
Loading file: Metro_invt_raw_all_homes_monthly.csv
